# Chapter 9 トピック

In [1]:
import common_func as cf

## 9.1 潜在的意味インデキシング

In [2]:
# data/ch05にある各文書から文書ベクトルを作成する
book_texts = [cf.get_string_from_file('irpb-files/data/ch05/%d.txt' % i) for i in range(10)]
tfidf_model,dic,tfidf_weights = cf.get_tfidfmodel_and_weights(book_texts)


In [3]:
# LSIモデルの作成
from gensim.models import LsiModel

# トピックの数
num_topics = 5

# モデルの作成
lsi_model = LsiModel(corpus=tfidf_weights,id2word=dic,num_topics=num_topics)

In [7]:
# トピックの特徴語の表示
print(lsi_model.print_topic(0,3))

0.624*"知能" + 0.484*"人工" + 0.204*"マービン・ミンスキー"


In [4]:
#lsi_search関数
def lsi_search(texts,query,num_topics):
    
    from gensim.similarities import MatrixSimilarity
    
    # tfidfに基づいて語の重みを計算する
    tfidf_model,dic,text_tfidf_weights = cf.get_tfidfmodel_and_weights(texts)
    
    # LSIモデルを作成し、トピックの重みを計算する
    lsi_model = LsiModel(corpus=text_tfidf_weights,id2word=dic,num_topics=num_topics)
    lsi_weights = lsi_model[text_tfidf_weights]
    index = MatrixSimilarity(lsi_weights,num_features=len(dic))
    
    # queryのbag-of-wordsを作成し、重みを計算する
    query_bows = cf.get_bows([query],dic)
    query_tfidf_weights = cf.get_weights(query_bows,dic,tfidf_model)
    query_lsi_weights = lsi_model[query_tfidf_weights]
    
    # 類似度計算
    sims = index[query_lsi_weights[0]]
    
    # 類似度で降順にソートする
    return sorted(enumerate(sims),key=lambda x: x[1],reverse=True)

In [17]:
# vsm_searchによる検索(再掲)
from pprint import pprint

query = '人工知能'

# tfidfモデルに基づく検索(リスト5.2と同じ)
# 検索結果は関連殿高さで順位付けされている
tfidf_result = cf.vsm_search(book_texts,query)

# 文書番号と関連度の組を出力する
pprint(tfidf_result)

[(1, 0.7481151),
 (8, 0.6076249),
 (5, 0.31722325),
 (0, 0.21160641),
 (2, 0.18004589),
 (3, 0.0),
 (4, 0.0),
 (6, 0.0),
 (7, 0.0),
 (9, 0.0)]


In [18]:
# lsi_searchによる検索
num_topics = 5

query = '人工知能'

lsi_result = lsi_search(book_texts,query,num_topics)
pprint(lsi_result)

[(8, 0.99998426),
 (1, 0.99996907),
 (5, 0.9991018),
 (0, 0.5014957),
 (2, 0.40058395),
 (9, 0.0017561095),
 (3, 0.0),
 (6, 0.0),
 (7, 0.0),
 (4, -0.0029632207)]


In [19]:
# TF/IDFとLSIの検索性能比較

# '人工知能'をクエリとする検索の正解
right_answer = [0,1,0,1,0,1,0,0,1,1]

# ランキングの計算
tfidf_ranking = [x[0] for x in tfidf_result]
lsi_ranking = [x[0] for x in lsi_result]

# 平均適合率による検索性能評価の比較
print('TFIDF: %.4f' % cf.get_average_precision(tfidf_ranking,right_answer))
print('LSI:   %.4f' % cf.get_average_precision(lsi_ranking,right_answer))

TFIDF: 0.8211
LSI:   0.8648


## 9.2 非負値行列因子分解

In [6]:
# nmf_search関数
from gensim.models.nmf import Nmf
from gensim.similarities import MatrixSimilarity

def nmf_search(texts,query,num_topics,passes=20,random_state=None):
    tfidf_model,dic,text_tfidf_weights = cf.get_tfidfmodel_and_weights(texts)
    
    # NMFモデルを作成
    nmf_model = Nmf(corpus=text_tfidf_weights,id2word=dic,num_topics=num_topics,passes=passes,random_state=random_state)
    
    # TF/IDFによる文書ベクトルをトピックベースのベクトルに変換
    nmf_weights = nmf_model[text_tfidf_weights]
    
    index = MatrixSimilarity(nmf_weights,num_features=len(dic))
    
    query_bows = cf.get_bows([query],dic)
    query_tfidf_weights = cf.get_weights(query_bows,dic,tfidf_model)
    query_nmf_weights = nmf_model[query_tfidf_weights]
    
    sims = index[query_nmf_weights[0]]
    return sorted(enumerate(sims),key=lambda x: x[1],reverse=True)

In [10]:
# nmf_searchによる検索
# トピック数を５に設定
from pprint import pprint

num_topics = 5
query = '人工知能'

# book_texts,queryはこれまでと同じ
# 結果を再現するためにrandom_stateを設定する
nmf_result = nmf_search(book_texts,query,num_topics,random_state=7)
pprint(nmf_result)

# NMFに基づく検索の性能評価
right_answer = [0,1,0,1,0,1,0,0,1,1]

nmf_ranking = [x[0] for x in nmf_result]
print('%.4f' % cf.get_average_precision(nmf_ranking,right_answer))

[(5, 0.9901341),
 (1, 0.9872562),
 (8, 0.96037483),
 (9, 0.74584293),
 (0, 0.13932848),
 (7, 0.13932848),
 (2, 0.07689431),
 (4, 0.008907776),
 (6, 0.0014151302),
 (3, 0.0)]
0.8944


## 9.3 潜在的ディリクレ配分法

In [13]:
# LDAモデルの作成
from gensim.models import LdaModel
from pprint import pprint

# LDAモデルの作成(この処理の中でトピックが計算される)
# dic,tfidf_weightsなどはリスト9.1で定義されたもの
# 再現性をもたせるためrandom_stateに特定の値(6)を設定する
lda_model = LdaModel(corpus=tfidf_weights,id2word=dic,num_topics=5,passes=20,random_state=6)

# 文書ベクトルからトピックの分布を計算
lda_weights = lda_model[tfidf_weights]

# 1番文書の内容を表示する
print(book_texts[0])

# 1番文書のトピックの確率分布を表示する
pprint(lda_weights[0])

パームアイランドはドバイの沖合に造られた人工島群で，ヤシの木をモチーフにデザインされています．この人工島には大型商業施設やホテル，別荘やなどが建てられています．

[(0, 0.043922454),
 (1, 0.825511),
 (2, 0.043527253),
 (3, 0.04351666),
 (4, 0.04352258)]


In [14]:
# トピックの特徴語の表示
# 0番トピックの確率分布のうち上位4語を表示
print(lda_model.print_topic(0,4))

0.041*"知能" + 0.030*"人工" + 0.021*"ネットワーク" + 0.017*"マービン・ミンスキー"


In [15]:
# lda_search関数
def lda_search(texts,query,num_topics,passes=20,random_state=None):
    tfidf_model,dic,text_tfidf_weights = cf.get_tfidfmodel_and_weights(texts)

    from gensim.similarities import MatrixSimilarity

    # LDAモデルを作成
    lda_model = LdaModel(corpus=text_tfidf_weights,id2word=dic,num_topics=num_topics,passes=passes,random_state=random_state)
    lda_weights = lda_model[text_tfidf_weights]
    index = MatrixSimilarity(lda_weights,num_features=len(dic))
    
    query_bows = cf.get_bows([query],dic)
    query_tfidf_weights = cf.get_weights(query_bows,dic,tfidf_model)
    query_lda_weights = lda_model[query_tfidf_weights]
    
    sims = index[query_lda_weights[0]]
    return sorted(enumerate(sims),key=lambda x: x[1],reverse=True)

In [16]:
# LDAに基づく検索の性能評価
# トピック数を5に設定
num_topics = 5
right_answer = [0,1,0,1,0,1,0,0,1,1]
query = '人工知能'

lda_result = lda_search(book_texts,query,num_topics,random_state=6)
lda_ranking = tuple([x[0] for x in lda_result])
print('%.4f' % cf.get_average_precision(lda_ranking,right_answer))

0.9633


In [17]:
# LDAに基づく検索の性能評価(複数回)
# トピック数を5に設定
num_topics = 5

# 検索の試行回数を5に設定
num_trials = 5
sum_of_ap = 0.0

right_answer = [0,1,0,1,0,1,0,0,1,1]
query = '人工知能'

for i in range(num_trials):
    lda_result = lda_search(book_texts,query,num_topics)
    lda_ranking = tuple([x[0] for x in lda_result])
    ap = cf.get_average_precision(lda_ranking,right_answer)
    print('%d: %.4f' % (i,ap))
    sum_of_ap += ap
print('平均: %.4f' % (sum_of_ap / num_trials))

0: 0.9183
1: 0.9183
2: 0.7563
3: 0.5677
4: 0.7514
平均: 0.7824
